# Prepare training container

In [ ]:
# loging to Sagemaker ECR with Deep Learning Containers
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-2.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 553020858742.dkr.ecr.us-east-2.amazonaws.com

In [ ]:
! pygmentize -l docker Dockerfile

In [96]:
! ./build_and_push.sh hf-transformers-sm latest Dockerfile

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon    235kB
Step 1/10 : FROM 763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-training:1.5.0-gpu-py36-cu101-ubuntu16.04
 ---> 47cd15520b75
Step 2/10 : LABEL author="vadimd@amazon.com"
 ---> Using cache
 ---> 9b087555bcfc
Step 3/10 : WORKDIR /opt/ml/code
 ---> Using cache
 ---> c61703f87573
Step 4/10 : RUN git clone https://github.com/huggingface/transformers
 ---> Using cache
 ---> ba58d750198b
Step 5/10 : RUN cd transformers/ &&     python3 -m pip install --no-cache-dir .
 ---> Using cache
 ---> b3f2b386ac2a
Step 6/10 : COPY container_training /opt/ml/code
 ---> Using cache
 ---> fd7a41991831
Step 7/10 : ENV SAGEMAKER_SUBMIT_DIRECTORY /

# Training Language Model

In [ ]:
# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [83]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.LocalSession() # can use LocalSession() to run container locally
# sess = sagemaker.Session()

bucket = sess.default_bucket()
region = "us-east-2"
account = sess.boto_session.client('sts').get_caller_identity()['Account']
prefix_input = 'transformer-input'
prefix_output = 'transformer-ouput'

In [ ]:
container = "hf-transformers-sm" # your container name
tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container, tag)

In [ ]:
# placeholder
lng_model_metrics=[]


In [84]:
lng_model_hp = {
    
    "nlp-problem":"language-modeling",
    "dataset" : "wiki",
    "do_train" : "true", # whether to train your model
    "do_eval" : "true",  # whether to run evaluation
    "fp16" : "true",     # using mixed precision

    #### Algo hyperparameters
    "model_type" : "gpt2",
    "model_name_or_path" : "gpt2",
    "per_gpu_train_batch_size" : "2",
    "per_gpu_eval_batch_size" : "2",
    "gradient_accumulation_steps" : "2",
}

In [97]:
lng_est = sagemaker.estimator.Estimator(image,
                                   role=role,
                                   train_instance_count=1, 
#                                    train_instance_type='ml.p3.16xlarge',
                                   train_instance_type="local_gpu", # use local_gpu for quick troubleshooting
#                                    train_volume_size=100,
                                   output_path="s3://{}/{}".format(sess.default_bucket(), prefix_output),
                                   metric_definitions = lng_model_metrics,
                                   hyperparameters = lng_model_hp, 
                                   sagemaker_session=sess)

lng_est.fit({"train":"s3://vadimd-nlp-datasets/wikitext-2-raw/wiki.train.raw", 
             "test":"s3://vadimd-nlp-datasets/wikitext-2-raw/wiki.test.raw"},
             job_name = "hf-transformers-batch2-fp16-v2",
             wait=False
           ) 

Creating tmpc2680p5v_algo-1-edk7g_1 ... 
Attaching to tmpc2680p5v_algo-1-edk7g_12mdone
algo-1-edk7g_1  | 2020-05-20 16:33:00,087 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-edk7g_1  | 2020-05-20 16:33:00,089 sagemaker-containers INFO     Failed to parse hyperparameter nlp-problem value language-modeling to Json.
algo-1-edk7g_1  | Returning the value itself
algo-1-edk7g_1  | 2020-05-20 16:33:00,089 sagemaker-containers INFO     Failed to parse hyperparameter dataset value wiki to Json.
algo-1-edk7g_1  | Returning the value itself
algo-1-edk7g_1  | 2020-05-20 16:33:00,089 sagemaker-containers INFO     Failed to parse hyperparameter model_type value gpt2 to Json.
algo-1-edk7g_1  | Returning the value itself
algo-1-edk7g_1  | 2020-05-20 16:33:00,089 sagemaker-containers INFO     Failed to parse hyperparameter model_name_or_path value gpt2 to Json.
algo-1-edk7g_1  | Returning the value itself
algo-1-edk7g_1  | 2020-05-20 16:33:00,166 sagemake

KeyboardInterrupt: 

# Train for GLUE benchmarks

In [ ]:
# Download script for GLUE data download
!wget https://gist.githubusercontent.com/W4ngatang/60c2bdb54d156a41194446737ce03e2e/raw/17b8dd0d724281ed7c3b2aeeda662b92809aadd5/download_glue_data.py


In [ ]:
! python download_glue_data.py

In [ ]:
! ./upload_data_to_s3.sh vadimd-nlp-datasets glue glue_data

In [79]:
text_cls_hp = {
    "nlp-problem" : "text-classification",
    "do_train" : "true", # whether to train your model
    "do_eval" : "true",  # whether to run evaluation
    
    #### Algo hyperparameters
    "task_name" : "MRPC", 
    "model_type" : "bert",
    "model_name_or_path" : "bert-base-cased", # provide only model name
    "max_seq_length" : 128,
    "per_gpu_train_batch_size" : 32,
    "learning_rate" : 2e-5,
    "num_train_epochs" : 3.0
}

In [ ]:
# placeholder
text_cls_metrics=[]

In [81]:
cls_est = sagemaker.estimator.Estimator(image,
                                   role=role,
                                   train_instance_count=1, 
                                   train_instance_type='ml.p3.2xlarge',
#                                   train_instance_type="local_gpu", # use local_gpu for quick troubleshooting
#                                   train_volume_size=100,
                                   output_path="s3://{}/{}".format(sess.default_bucket(), prefix_output),
                                   metric_definitions = text_cls_metrics,
                                   hyperparameters = text_cls_hp, 
                                   sagemaker_session=sess)

# TODO: prepare data channels
cls_est.fit({"train":"s3://vadimd-nlp-datasets/glue/MRPC"},
            job_name = "hf-transformers-glue-v2",
            wait=False) 

#  s3://vadimd-nlp-datasets/glue/MNLI/train.tsv